In [1]:
import pandas as pd
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
import math
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from keras.models import load_model
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils import to_categorical
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
df=pd.read_csv("emails.csv")
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [3]:
df['spam'].value_counts()

0    4360
1    1368
Name: spam, dtype: int64

In [7]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
print(stopwords.words('english'))
stop_words = set(stopwords.words('english'))

def rem_subject(text):
    return text[9:]

def remove_stop_words(sentence):
  if isinstance(sentence, str):
    words = sentence.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)
  else:
    return ""

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    if isinstance(text, str):
        st = ""
        for w in w_tokenizer.tokenize(text):
            st = st + lemmatizer.lemmatize(w) + " "
        return st.strip()
    else:
        return ""

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df['text']=df['text'].apply(rem_subject)
df['text']=df['text'].apply(remove_stop_words)
df['text']=df['text'].apply(lemmatize_text)

In [9]:
df.head()

,text,spam
0,irresistible corporate identity lt really hard...,1
1,ding gunslinger fanny merrill muzo colza attai...,1
2,ble new home made easy im wanting show homeown...,1
3,rinting special request additional information...,1
4,et software cd ! software compatibility . . . ...,1


In [10]:
def Cavg(text):
    arr=[]
    for i in text:
        np.array(arr.append(len(i)))
    return(np.average(arr))
avg_len=Cavg(df['text'])
print(avg_len)

1162.3261173184358


In [11]:
max_word=5000 #the maximum number of words to keep, based on word frequency.
max_sequence_length = 1000 #the average length was 96 so we can round of to 100
tokenizer = Tokenizer(num_words=max_word, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X= pad_sequences(X, maxlen=max_sequence_length)

In [12]:
y = keras.utils.to_categorical(df['spam'], 2, dtype="float32")
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4009, 1000)
(4009, 2)
(1719, 1000)
(1719, 2)


In [14]:
model = Sequential()
model.add(Embedding(max_word, 1000, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, dropout=0.5, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=0.5)))
model.add(Dense(2, activation='softmax'))
# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

126/126 [==============================] - 2242s 18s/step - loss: 0.1565 - accuracy: 0.9431 - val_loss: 0.0437 - val_accuracy: 0.9837


In [16]:
y_pred=model.predict(X_test)
report = classification_report(y_test.argmax(axis=1), np.around(y_pred, decimals=0).argmax(axis=1))
print("\nClassification Report:")
print(report)

54/54 [==============================] - 190s 3s/step

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1278
           1       0.94      1.00      0.97       441

    accuracy                           0.98      1719
   macro avg       0.97      0.99      0.98      1719
weighted avg       0.98      0.98      0.98      1719



In [17]:
import pickle
# saving deep learning model
model.save("EmailClassifier.h5")
# saving tokenizer
with open('tokenizer_email.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
with open('tokenizer_email.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
loaded_model=keras.models.load_model('EmailClassifier.h5')

In [21]:
max_sequence_length = 1000
label=['Not Spam', 'Spam']

In [31]:
test_txt=["Hi James, Have you claim your complimentary gift yet? I've compiled in here a special astrology gift that predicts everything about you in the future? This is your enabler to take the correct actions now. >> Click here to claim your copy now >> Claim yours now, and thank me later. Love, Heather", "Hey mate! i am mailing you regarding the opening at our company. You applied here for the role of a data scientist an we would like you to share your resume. regards, Manager"]
seq = tokenizer.texts_to_sequences(test_txt)
ans = pad_sequences(seq, maxlen=max_sequence_length)
preds = model.predict(ans)
print(preds)
for i in preds:
    print(label[np.around(i, decimals=0).argmax()])

1/1 [==============================] - 1s 1s/step
[[0.00361768 0.99638236]
 [0.98485434 0.01514569]]
Spam
Not Spam
